In [81]:
import numpy as np
from scipy.stats import multivariate_normal
from scipy.special import logsumexp
import matplotlib.pyplot as plt

# **1) GENERATION DES DONNEES**

In [82]:
def generate_z(A,b,K,cov,x):
    z_O = []
    z_E = []
    for i in range(int(2**K)):
        z_O.append(np.random.multivariate_normal(A*x+b, (2/3)*cov))
        z_E.append(np.random.multivariate_normal(A*x+b, (2/3)*cov))
    z = z_O.copy()
    z.extend(z_E)
    return z, z_O, z_E

In [83]:
def generate_q_p(A,b,K,mean,cov,x,z,z_O,z_E):
    q = []
    p = []
    q_O = []
    p_O = []
    q_E = []
    p_E = []
    for i in range(int(2**(K+1))):
        q.append(multivariate_normal.pdf(z[i], mean=A*x+b, cov=(2/3)*cov))
        p.append(multivariate_normal.pdf(z[i], mean=mean, cov=cov)*multivariate_normal.pdf(x, mean=z[i], cov=cov))
    for i in range(int(2**(K))):
        q_O.append(multivariate_normal.pdf(z_O[i], mean=A*x+b, cov=(2/3)*cov))
        p_O.append(multivariate_normal.pdf(z_O[i], mean=mean, cov=cov)*multivariate_normal.pdf(x, mean=z_O[i], cov=cov))
        q_E.append(multivariate_normal.pdf(z_E[i], mean=A*x+b, cov=(2/3)*cov))
        p_E.append(multivariate_normal.pdf(z_E[i], mean=mean, cov=cov)*multivariate_normal.pdf(x, mean=z_E[i], cov=cov))
    return q, p, q_O, p_O, q_E, p_E

In [84]:
x = np.random.multivariate_normal(np.zeros(20), 2*np.identity(20))
z, z_O, z_E = generate_z(1,0,5,np.identity(20),x)
q, p, q_O, p_O, q_E, p_E = generate_q_p(1,0,5,np.zeros(20), np.identity(20),x,z,z_O,z_E)
w = np.log(p) - np.log(q)

# **2) FONCTIONS**

In [85]:
def sum_w(K,a):
    sum_w=0
    for i in range(int(K)):
        sum_w += np.exp(w[i])
    return sum_w

In [86]:
def pi(K,w,p):
    sum_grad = 0
    for i in range(int(K)):
        sum_grad += (w[i]/sum_w(K,w))*np.gradient(p)[i]
    return sum_grad

In [91]:
def grad_estimator(n_sim, mean, cov, A, b, r, theta):
    grad_vector = []
    x = np.random.multivariate_normal(np.zeros(20)+theta, 2*cov)
    for i in range(n_sim):
        K = np.random.geometric(r, 1)
        z, z_O, z_E = generate_z(A,b,K,cov,x)
        q, p, q_O, p_O, q_E, p_E = generate_q_p(A,b,K,mean,cov,x,z,z_O,z_E)
        w = np.log(p) - np.log(q)
        w_O = np.log(p_O) - np.log(q_O)
        w_E = np.log(p_E) - np.log(q_E)
            
        #third step : compute I0
        I0 = w.mean()

        #fourth step : compute Lo and Le
            # Lo
        pi_O = pi(K,w_O,p_O)  
            #Le
        pi_E = pi(K,w_E,p_E)    

        #fifth step : compute L_O_E
        pi_O_E = pi(K,w,p)
    
        #DeltaK
        DeltaK = pi_O_E - 0.5*(pi_O + pi_E)
    
        #sixth step : compute the final estimator
        grad = I0 + (DeltaK/(r*(1-r)**(int(K)-1)))
        
        grad_vector.append(grad)
    mean_value = np.mean(grad_vector)
    std_error_value = np.std(grad_vector) / np.sqrt(len(grad_vector))

    return mean_value

In [92]:
theta = np.random.normal(0,1)
print(theta)
grad_estimator(100, np.zeros(20), np.identity(20), 1, 0, 0.6, theta)

-0.8098216560702469


-47.28695169816283

# **3) FONCTION POUR PLOT LE GRADIENT ESTIME**

In [93]:
def plot_grad(debut,fin,pas,n_sim):
    theta = np.random.normal(0,1)
    print(theta)
    for i in range(n_sim):
        liste = np.linspace(debut,fin,pas)
        x = []
        y = []
        for k in liste:
            y.append(grad_estimator(100, np.zeros(20)+k, np.identity(20), 1, 0, 0.6, theta))
            x.append(k)
        print((i+1)/n_sim) 
    return x, y

In [ ]:
x,y = plot_grad(-4,4,30,3)
plt.scatter(x,y)

0.6567058167107229
0.3333333333333333
0.6666666666666666
